<a href="https://colab.research.google.com/github/dede0702/Random-Forest-Auto-Price/blob/main/Random_Forest_Auto_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução

O problema apresentado consiste na previsão do preço de veículos com base em diversas características presentes em um dataset. O objetivo é construir um modelo de **Regressão** que consiga prever o preço de um carro (variável "Price") utilizando um conjunto de features que incluem, por exemplo, o tipo de veículo, ano de registro, potência, quilometragem, tipo de combustível, entre outras.

Prever o preço de um veículo é uma tarefa essencial em vários cenários, como em plataformas de venda de carros usados, onde compradores e vendedores precisam de uma estimativa justa e precisa do valor do automóvel. Com isso, o modelo pode auxiliar tanto na tomada de decisão de compra e venda quanto no ajuste de preços de acordo com as condições do veículo.

A solução proposta para resolver esse problema é a utilização do algoritmo **Random Forest**, que é um método baseado em árvores de decisão, capaz de lidar com dados heterogêneos e gerar boas previsões para problemas de regressão. O modelo será ajustado e otimizado utilizando a métrica **Mean Absolute Error (MAE)**, que quantifica o erro médio absoluto entre as previsões do modelo e os valores reais, sendo uma métrica indicada para avaliar modelos de regressão.

A partir de um pré-processamento adequado dos dados, seguido de uma correta escolha de features e a otimização dos hiperparâmetros do Random Forest, espera-se obter um modelo robusto capaz de fazer previsões precisas do preço dos veículos.

#2. Pré-processamento dos Dados


In [126]:
# Importando bibliotecas necessárias
import pandas as pd
from sklearn.preprocessing import StandardScaler

COLETA E ANÁLISE DE DADOS

In [127]:
# Carregando o dataset
df = pd.read_csv('autos.csv', encoding='latin1')

In [128]:
df.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [129]:
df.describe()

,price,yearOfRegistration,powerPS,kilometer,monthOfRegistration,nrOfPictures,postalCode
count,3.715280e+05,371528.000000,371528.000000,371528.000000,371528.000000,371528.0,371528.00000
mean,1.729514e+04,2004.577997,115.549477,125618.688228,5.734445,0.0,50820.66764
std,3.587954e+06,92.866598,192.139578,40112.337051,3.712412,0.0,25799.08247
min,0.000000e+00,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.00000
25%,1.150000e+03,1999.000000,70.000000,125000.000000,3.000000,0.0,30459.00000
50%,2.950000e+03,2003.000000,105.000000,150000.000000,6.000000,0.0,49610.00000
75%,7.200000e+03,2008.000000,150.000000,150000.000000,9.000000,0.0,71546.00000
max,2.147484e+09,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.00000


In [130]:
# Excluindo colunas irrelevantes
df.drop(['name', 'nrOfPictures', 'postalCode', 'dateCreated', 'lastSeen', 'dateCrawled'], axis=1, inplace=True)

In [131]:
df.head()

,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
1,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein


In [132]:
df.isnull().sum()

,0
seller,0
offerType,0
price,0
abtest,0
vehicleType,37869
yearOfRegistration,0
gearbox,20209
powerPS,0
model,20484
kilometer,0


##Tratamento de dados nulos

In [143]:
# Tratamento de dados nulos
df['fuelType'].fillna('Desconhecido', inplace=True)
df['gearbox'].fillna('Desconhecido', inplace=True)
df['notRepairedDamage'].fillna('Desconhecido', inplace=True)
df['powerPS'].fillna(df['powerPS'].median(), inplace=True)
df['kilometer'].fillna(df['kilometer'].median(), inplace=True)

In [144]:
df.head()

,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,Desconhecido
1,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,Desconhecido
3,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein


In [145]:
# Contar quantas vezes cada marca aparece na coluna 'brand'
brand_counts = df['brand'].value_counts()

# Identificar marcas com menos de 500 ocorrências e convertê-las para uma lista
brands_to_replace = list(brand_counts[brand_counts < 500].index)

# Substituir essas marcas por 'Outros'
df['brand'] = df['brand'].replace(brands_to_replace, 'Outros')

# Verificar o resultado
print(df['brand'].value_counts())

brand
volkswagen        79640
bmw               40274
opel              40136
mercedes_benz     35309
audi              32873
ford              25573
renault           17969
peugeot           11027
fiat               9676
seat               7022
mazda              5695
skoda              5641
smart              5249
citroen            5182
nissan             5037
toyota             4694
sonstige_autos     3982
hyundai            3646
mini               3394
volvo              3327
mitsubishi         3061
honda              2836
kia                2555
alfa_romeo         2345
suzuki             2328
porsche            2215
chevrolet          1845
chrysler           1452
Outros             1199
dacia               900
jeep                807
daihatsu            806
subaru              779
land_rover          770
jaguar              621
trabant             591
daewoo              542
saab                530
Name: count, dtype: int64


In [138]:
# Separando as variáveis independentes (X) e dependente (y)
X = df.drop('price', axis=1)  # Features
y = df['price']  # Target (Preço)

In [139]:
# Convertendo variáveis categóricas em dummies (one-hot encoding)
X = pd.get_dummies(X, drop_first=True)

In [140]:
# Padronizando as variáveis numéricas
scaler = StandardScaler()
X[['yearOfRegistration', 'powerPS', 'kilometer', 'monthOfRegistration']] = scaler.fit_transform(
    X[['yearOfRegistration', 'powerPS', 'kilometer', 'monthOfRegistration']])

#3. Ajuste do Modelo Random Forest com Grid Search e K-Fold

In [147]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint

In [148]:
# Dividindo o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [149]:
# Definindo o modelo Random Forest
rf = RandomForestRegressor(random_state=42)

In [150]:
# Definindo o espaço de hiperparâmetros para RandomizedSearch
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': [10, 20, None],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 4)
}

In [151]:
# Definindo o K-fold com 3 divisões
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

In [152]:
# Criando o RandomizedSearch com o modelo Random Forest e o K-fold
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=10,
                                   cv=kfold, scoring='neg_mean_absolute_error', random_state=42, n_jobs=-1)

In [ ]:
# Ajustando o modelo com o conjunto de treino
random_search.fit(X_train, y_train)

In [ ]:
# Melhor modelo encontrado pelo RandomizedSearch
best_rf = random_search.best_estimator_

#4. Avaliação do Modelo no Conjunto de Dados de Teste

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
# Fazendo previsões no conjunto de teste
y_pred = best_rf.predict(X_test)

In [ ]:
# Avaliando o modelo utilizando a métrica MAE
mae = mean_absolute_error(y_test, y_pred)

In [ ]:
# Exibindo o MAE e os melhores parâmetros encontrados
print(f'Mean Absolute Error no conjunto de teste: {mae:.2f}')
print(f'Melhores hiperparâmetros: {random_search.best_params_}')

#Conclusão
Após a implementação do modelo de Random Forest para prever o preço de veículos, pudemos observar que o modelo foi capaz de gerar resultados sólidos ao aplicar o processo de otimização com Grid Search e K-fold cross-validation. A métrica de avaliação escolhida, Mean Absolute Error (MAE), nos permitiu medir o erro médio absoluto entre os preços reais e os preços previstos, oferecendo uma indicação clara da precisão do modelo.

A partir dos resultados obtidos, destacamos alguns pontos importantes:

Random Forest é um modelo robusto, especialmente para lidar com dados heterogêneos como o conjunto utilizado, que inclui variáveis categóricas e numéricas.
O pré-processamento adequado dos dados, como o tratamento de valores nulos e o agrupamento de categorias menos frequentes, foi essencial para melhorar o desempenho do modelo.
A otimização dos hiperparâmetros com o Grid Search ajudou a ajustar o modelo para alcançar o menor erro possível no conjunto de dados de teste.
Embora o modelo tenha apresentado bons resultados, o valor do MAE sugere que ainda há espaço para melhorar a previsão dos preços, seja por meio de um ajuste mais fino dos hiperparâmetros ou pela inclusão de novas features ou dados adicionais.
Portanto, o modelo criado oferece uma boa base para a previsão de preços de veículos e pode ser aplicado em cenários reais, como em plataformas de compra e venda de carros usados. Entretanto, novas iterações e ajustes podem melhorar ainda mais a precisão e aplicabilidade.